In [1]:
!pip install hvac langchain==0.0.154 google-api-python-client openai python-dotenv

In [2]:
import hvac
import langchain
import os
import sys

from dotenv import load_dotenv
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, OpenAI
# from langchain.chat_models import ChatOpenAI
from langchain.utilities import (
    BashProcess,
    GoogleSearchAPIWrapper,
    PythonREPL,
    TextRequestsWrapper,
)

from sonarr_recent import recently_added_episodes
from solar_production import solar_production
from trending_news import trending_news

# Load the environment variables from the .env file
load_dotenv()

True

In [3]:
client = hvac.Client(
    url=os.getenv('VAULT_API'),
    token=os.getenv('VAULT_TOKEN'),
)

# print(client.is_authenticated())

In [4]:
try:
    secret_resp = client.secrets.kv.v2.read_secret_version(
        mount_point='kv', 
        path='langchain-bot', 
        raise_on_deleted_version=False
    )
    
    if secret_resp['data'] is not None:
        secret_values = secret_resp['data']['data']
        for secret, value in secret_values.items():
            os.environ[str(secret)] = str(value)
    else:
        print("The secret does not exist.")
except hvac.exceptions.InvalidPath:
    print("The path is invalid or the permission is denied.")
except hvac.exceptions.Forbidden:
    print("The permission is denied.")
except hvac.exceptions.VaultError as e:
    print(f"Vault error occurred: {e}")

In [41]:
bash = BashProcess()
python_repl = PythonREPL()
requests = TextRequestsWrapper()
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "Recent TV Show Episodes",
        func = recently_added_episodes,
        description = "Useful for when you need to understand what TV show episodes were recently added to Plex; input should be 'test'; output is returned as JSON"
    ),
    Tool(
        name = "Solar Power Production",
        func = solar_production,
        description = "Useful for when you need to understand today's solar power production; input should be 'test'; output is returned as a string",
        return_direct = True
    ),
    Tool(
        name = "Trending News",
        func = trending_news,
        description = "Useful for when you need to understand the current trending news topics; input should be 'test'; output is returned as a Python list",
        return_direct = True
    ),
    Tool(
        name = "Bash",
        func = bash.run,
        description = "Useful for when you need to run a shell command locally",
    ),
    Tool(
        name = "Python",
        func = python_repl.run,
        description = "Useful for when you need to run a Python command locally",
    ),
    Tool(
        name = "Requests",
        func = requests.get,
        description = "Useful for when you need to access a website or URL",
    ),
    Tool(
        name = "Search",
        func = search.run,
        description = "Useful for when you need to access the internet for additional information"
    ),
]

In [42]:
prefix = """Have a conversation answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [43]:
# Initialize Chat Memory
memory = ConversationBufferMemory(memory_key="chat_history")

In [44]:
# Setup ChatGPT Agent
llm_chain = LLMChain(
    llm=OpenAI(
        temperature=0.3,
        model_name="gpt-3.5-turbo", # gpt-3.5-turbo OR gpt-4
        request_timeout=60,
        max_retries=3,
    ),
    prompt=prompt,
)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    memory=memory,
    tools=tools,
    verbose=True,
)

In [45]:
def chat_completion(input):
    try:
        output = agent_chain.run(input=input)
        return output
    except Exception as e:
        print(f"🦜🔗 LangChain Error: {e}")
        return e

In [46]:
%%time
resp = chat_completion("How much solar power was generated?")
print(resp)



> Entering new AgentExecutor chain...
Thought: I need to check the solar power production data
Action: Solar Power Production
Action Input: 'test'
Observation: Today's Solar Production: 37.3 kWh

> Finished chain.
Today's Solar Production: 37.3 kWh
CPU times: user 4.5 ms, sys: 4.67 ms, total: 9.17 ms
Wall time: 2.33 s
